In [ ]:
# | default_exp icdar_data

In [ ]:
# | export
import os
import re
import shutil
import tempfile
from dataclasses import dataclass
from pathlib import Path
from typing import Dict, List
from typing import Text as TypingText
from typing import Tuple
from zipfile import ZipFile

import edlib
import pandas as pd
from loguru import logger
from tqdm import tqdm


The [ICDAR 2019 Competition on Post-OCR Text Correction 
dataset](https://sites.google.com/view/icdar2019-postcorrectionocr/dataset) 
([zenodo record](https://zenodo.org/record/3515403#.YwULoi0RoWI))
contains text files in the following format:

```
[OCR_toInput] This is a cxample...
[OCR_aligned] This is a@ cxample...
[ GS_aligned] This is an example.@@
01234567890123
```

The first line contains the ocr input text. The second line contains the aligned 
ocr and the third line contains the aligned gold standard.
`@` is the aligment character and `#` represents characters in the OCR that do 
not occur in the gold standard.

For working with this data, the first 14 characters have to be removed. We also 
remove leading and trailing whitespace.

In [ ]:
# | export
def remove_label_and_nl(line: str):
    return line.strip()[14:]

In [ ]:
# | hide
assert (
    remove_label_and_nl("[OCR_toInput] This is a cxample...\n")
    == "This is a cxample..."
)

## Tokenization

Task 1 of the competition is about finding tokens with OCR mistakes. In this context a token
refers to a string between two whitespaces.

In [ ]:
# | export
@dataclass
class AlignedToken:
    """Dataclass for storing aligned tokens"""

    ocr: str  # String in the OCR text
    gs: str  # String in the gold standard
    ocr_aligned: str  # String in the aligned OCR text (without aligmnent characters)
    gs_aligned: str  # String in the aligned GS text (without aligmnent characters)
    start: int  # The index of the first character in the OCR text
    len_ocr: int  # The lentgh of the OCR string

In [ ]:
# | export
def tokenize_aligned(ocr_aligned: str, gs_aligned: str) -> List[AlignedToken]:
    """Get a list of AlignedTokens from the aligned OCR and GS strings"""

    ocr_cursor = 0
    start = 0

    ocr_token_chars: List[str] = []
    gs_token_chars: List[str] = []
    ocr_token_chars_aligned: List[str] = []
    gs_token_chars_aligned: List[str] = []

    tokens = []

    for ocr_aligned_char, gs_aligned_char in zip(ocr_aligned, gs_aligned):
        # print(ocr_aligned_char, gs_aligned_char, ocr_cursor)
        # The # character in ocr is not an aligment character!
        if ocr_aligned_char != "@":
            ocr_cursor += 1

        if ocr_aligned_char == " " and gs_aligned_char == " ":
            # print('TOKEN')
            # print('OCR:', repr(''.join(ocr_token_chars)))
            # print(' GS:', repr(''.join(gs_token_chars)))
            # print('start:', start_char)
            # ocr_cursor += 1

            # Ignore 'tokens' without representation in the ocr text
            # (these tokens do not consist of characters)
            ocr = ("".join(ocr_token_chars)).strip()
            if ocr != "":
                tokens.append(
                    AlignedToken(
                        ocr,
                        "".join(gs_token_chars),
                        "".join(ocr_token_chars_aligned),
                        "".join(gs_token_chars_aligned),
                        start,
                        len("".join(ocr_token_chars)),
                    )
                )
            start = ocr_cursor

            ocr_token_chars = []
            gs_token_chars = []
            ocr_token_chars_aligned = []
            gs_token_chars_aligned = []
        else:
            ocr_token_chars_aligned.append(ocr_aligned_char)
            gs_token_chars_aligned.append(gs_aligned_char)
            # The # character in ocr is not an aligment character!
            if ocr_aligned_char != "@":
                ocr_token_chars.append(ocr_aligned_char)
            if gs_aligned_char != "@" and gs_aligned_char != "#":
                gs_token_chars.append(gs_aligned_char)

    # Final token (if there is one)
    ocr = ("".join(ocr_token_chars)).strip()
    if ocr != "":
        tokens.append(
            AlignedToken(
                ocr,
                "".join(gs_token_chars),
                "".join(ocr_token_chars_aligned),
                "".join(gs_token_chars_aligned),
                start,
                len("".join(ocr_token_chars)),
            )
        )

    return tokens

In [ ]:
tokenize_aligned("This is a@ cxample...", "This is an example.##")

[AlignedToken(ocr='This', gs='This', ocr_aligned='This', gs_aligned='This', start=0, len_ocr=4),
 AlignedToken(ocr='is', gs='is', ocr_aligned='is', gs_aligned='is', start=5, len_ocr=2),
 AlignedToken(ocr='a', gs='an', ocr_aligned='a@', gs_aligned='an', start=8, len_ocr=1),
 AlignedToken(ocr='cxample...', gs='example.', ocr_aligned='cxample...', gs_aligned='example.##', start=10, len_ocr=10)]

The OCR text of an AlignedToken may still consist of multiple tokens. This is the 
case when the OCR text contains one or more spaces. To make sure the (sub)tokenization
of a token is the same, no matter if it was not yet tokenized completely, 
another round of tokenization is added.

In [ ]:
# | export
@dataclass
class InputToken:
    """Dataclass for the tokenization within AlignedTokens"""

    ocr: str
    gs: str
    start: int
    len_ocr: int
    label: int

In [ ]:
# | export
def leading_whitespace_offset(string: str) -> int:
    """
    Return the leading whitespace offset for an aligned ocr string

    If an aligned ocr string contains leading whitespace, the offset needs to be added
    to the start index of the respective input tokens.

    Args:
        string (str): aligned ocr string to determine the leading whitespace offset for

    Returns:
        int: leading whitespace offset for input tokens
    """
    offset = 0

    regex = r"^@*(?P<whitespace>\ +)"
    match = re.match(regex, string)
    if match:
        offset = len(match.group("whitespace"))

    return offset

In [ ]:
# | hide
string = "bla"

assert leading_whitespace_offset(string) == 0

In [ ]:
# | hide
string = " bla"

assert leading_whitespace_offset(string) == 1

In [ ]:
# | hide
string = "@ bla"

assert leading_whitespace_offset(string) == 1

In [ ]:
# | hide
string = "@  bla"

assert leading_whitespace_offset(string) == 2

In [ ]:
# | export
def get_input_tokens(aligned_token: AlignedToken):
    """Tokenize an AlignedToken into subtokens and assign task 1 labels"""
    input_tokens = []
    if aligned_token.ocr == aligned_token.gs:
        input_tokens.append(
            InputToken(
                ocr=aligned_token.ocr,
                gs=aligned_token.gs,
                start=aligned_token.start,
                len_ocr=len(aligned_token.ocr),
                label=0,
            )
        )
    else:
        parts = aligned_token.ocr.split(" ")
        new_start = aligned_token.start
        offset = leading_whitespace_offset(aligned_token.ocr_aligned)
        for i, part in enumerate(parts):
            if i == 0:
                token = InputToken(
                    ocr=part,
                    gs=aligned_token.gs,
                    start=aligned_token.start,
                    len_ocr=len(part),
                    label=1,
                )
            else:
                token = InputToken(
                    ocr=part,
                    gs="",
                    start=new_start,
                    len_ocr=len(part),
                    label=2,
                )
                token.start += offset
            new_start += len(part) + 1
            input_tokens.append(token)
    return input_tokens

In [ ]:
t = AlignedToken("Major", "Major", "Major", "Major", 19, 5)
print(t)

for inp_tok in get_input_tokens(t):
    print(inp_tok)

AlignedToken(ocr='Major', gs='Major', ocr_aligned='Major', gs_aligned='Major', start=19, len_ocr=5)
InputToken(ocr='Major', gs='Major', start=19, len_ocr=5, label=0)


In [ ]:
# | hide
tokens = []
labels = []
gs = []

for inp_tok in get_input_tokens(t):
    tokens.append(inp_tok.ocr)
    labels.append(inp_tok.label)
    gs.append(inp_tok.gs)

assert tokens == ["Major"]
assert labels == [0]
assert "".join(gs) == t.gs

In [ ]:
t = AlignedToken("INEVR", "I NEVER", "I@NEV@R", "I NEVER", 0, 5)
print(t)

for inp_tok in get_input_tokens(t):
    print(inp_tok)

AlignedToken(ocr='INEVR', gs='I NEVER', ocr_aligned='I@NEV@R', gs_aligned='I NEVER', start=0, len_ocr=5)
InputToken(ocr='INEVR', gs='I NEVER', start=0, len_ocr=5, label=1)


In [ ]:
# | hide
tokens = []
labels = []
gs = []

for inp_tok in get_input_tokens(t):
    tokens.append(inp_tok.ocr)
    labels.append(inp_tok.label)
    gs.append(inp_tok.gs)

assert tokens == ["INEVR"]
assert labels == [1]
assert "".join(gs) == t.gs

In [ ]:
t = AlignedToken("Long ow.", "Longhow.", "Long ow.", "Longhow.", 24, 8)
print(t)

for inp_tok in get_input_tokens(t):
    print(inp_tok)

AlignedToken(ocr='Long ow.', gs='Longhow.', ocr_aligned='Long ow.', gs_aligned='Longhow.', start=24, len_ocr=8)
InputToken(ocr='Long', gs='Longhow.', start=24, len_ocr=4, label=1)
InputToken(ocr='ow.', gs='', start=29, len_ocr=3, label=2)


In [ ]:
# | hide
tokens = []
labels = []
gs = []

for inp_tok in get_input_tokens(t):
    tokens.append(inp_tok.ocr)
    labels.append(inp_tok.label)
    gs.append(inp_tok.gs)

assert tokens == ["Long", "ow."]
assert labels == [1, 2]
assert "".join(gs) == t.gs

In [ ]:
# | hide
# Example where only the second input token does not match (the first input token does
# match)
t = AlignedToken(
    ocr="1 2", gs="1", ocr_aligned="1 2", gs_aligned="1@@", start=324, len_ocr=3
)
print(t)

tokens = []
labels = []
gs = []

for inp_tok in get_input_tokens(t):
    print(inp_tok)
    tokens.append(inp_tok.ocr)
    labels.append(inp_tok.label)
    gs.append(inp_tok.gs)

assert tokens == ["1", "2"]
assert labels == [1, 2]
assert "".join(gs) == t.gs

AlignedToken(ocr='1 2', gs='1', ocr_aligned='1 2', gs_aligned='1@@', start=324, len_ocr=3)
InputToken(ocr='1', gs='1', start=324, len_ocr=1, label=1)
InputToken(ocr='2', gs='', start=326, len_ocr=1, label=2)


## Process a text file

Next, we need functions for processing a text in the ICDAR data format.

In [ ]:
# | export
@dataclass
class Text:
    """Dataclass for storing a text in the ICDAR data format"""

    ocr_text: str
    tokens: list
    input_tokens: list
    score: float

In [ ]:
# | export
def clean(string: str) -> str:
    """Remove alignment characters from a text"""
    string = string.replace("@", "")
    string = string.replace("#", "")

    return string

In [ ]:
# | export
def normalized_ed(ed: int, ocr: str, gs: str) -> float:
    """Returns the normalized editdistance"""
    score = 0.0
    longest = max(len(ocr), len(gs))
    if longest > 0:
        score = ed / longest
    return score

In [ ]:
# | export
def process_text(in_file: Path) -> Text:
    """Process a text from the ICDAR dataset

    Extract AlignedTokens, InputTokens, and calculate normalized editdistance.
    """
    with open(in_file) as f:
        lines = f.readlines()

    # The # character in ocr input is not an aligment character, but the @
    # character is!
    ocr_input = remove_label_and_nl(lines[0]).replace("@", "")
    ocr_aligned = remove_label_and_nl(lines[1])
    gs_aligned = remove_label_and_nl(lines[2])

    # print('ocr input:', ocr_input)
    # print('ocr aligned:', ocr_aligned)
    # print('gs aligned:',gs_aligned)

    tokens = tokenize_aligned(ocr_aligned, gs_aligned)

    # Check data
    for token in tokens:
        input_token = ocr_input[token.start : token.start + token.len_ocr]
        try:
            assert token.ocr == input_token.strip()
        except AssertionError:
            logger.warning(
                f"OCR != aligned OCR: Text: {str(in_file)}; ocr: {repr(token.ocr)}; "
                + f"ocr_input: {repr(input_token)}"
            )
            raise

    ocr = clean(ocr_aligned)
    gs = clean(gs_aligned)

    try:
        ed = edlib.align(gs, ocr)["editDistance"]
        score = normalized_ed(ed, ocr, gs)
    except UnicodeEncodeError:
        logger.warning(f"UnicodeEncodeError for text {in_file}; setting score to 1")
        score = 1

    input_tokens = []
    for token in tokens:
        for inp_tok in get_input_tokens(token):
            input_tokens.append(inp_tok)

    return Text(ocr_input, tokens, input_tokens, score)

Processing the example text:

In [ ]:
in_file = Path(os.getcwd()) / "data" / "example.txt"
text = process_text(in_file)
text

Text(ocr_text='This is a cxample...', tokens=[AlignedToken(ocr='This', gs='This', ocr_aligned='This', gs_aligned='This', start=0, len_ocr=4), AlignedToken(ocr='is', gs='is', ocr_aligned='is', gs_aligned='is', start=5, len_ocr=2), AlignedToken(ocr='a', gs='an', ocr_aligned='a@', gs_aligned='an', start=8, len_ocr=1), AlignedToken(ocr='cxample...', gs='example.', ocr_aligned='cxample...', gs_aligned='example.@@', start=10, len_ocr=10)], input_tokens=[InputToken(ocr='This', gs='This', start=0, len_ocr=4, label=0), InputToken(ocr='is', gs='is', start=5, len_ocr=2, label=0), InputToken(ocr='a', gs='an', start=8, len_ocr=1, label=1), InputToken(ocr='cxample...', gs='example.', start=10, len_ocr=10, label=1)], score=0.2)

In [ ]:
# | hide
assert len(text.tokens) == 4
assert len(text.input_tokens) == 4
assert text.tokens[2].ocr == "a"
assert text.score == 0.2

In [ ]:
# | hide
# Token missing from ocr input (original):
# [OCR_toInput] example &
# [OCR_aligned] example & @@@
# [ GS_aligned] example x bla

# Because we only get the OCR text, we can't know the actual length of the text

in_file = Path(os.getcwd()) / "data" / "example3.txt"

text = process_text(in_file)

assert len(text.tokens) == 2
assert len(text.input_tokens) == 2

## Process the entire dataset

File structure of the ICDAR dataset
```
.
├── <data_dir>
│   ├── <language>
│   │   ├── <language (set)>1
│   │   ...
│   │   └── <language (set)>n
│   ...
...
```

In [ ]:
# | export
def generate_data(in_dir: Path) -> Tuple[Dict[str, Text], pd.DataFrame]:
    """Process all texts in the dataset and return a dataframe with metadata"""

    data = {}

    file_languages = []
    file_subsets = []
    file_names = []
    scores = []
    num_tokens = []
    num_input_tokens = []

    for language_dir in tqdm(in_dir.iterdir()):
        for text_file in language_dir.rglob("*.txt"):
            # print(text_file)
            # print(text_file.relative_to(in_dir))
            key = str(text_file.relative_to(in_dir))
            data[key] = process_text(text_file)

            language, subset, _ = key.split(os.sep)

            file_languages.append(language)
            file_subsets.append(subset)
            file_names.append(key)
            scores.append(data[key].score)
            num_tokens.append(len(data[key].tokens))
            num_input_tokens.append(len(data[key].input_tokens))
    md = pd.DataFrame(
        {
            "language": file_languages,
            "subset": file_subsets,
            "file_name": file_names,
            "score": scores,
            "num_tokens": num_tokens,
            "num_input_tokens": num_input_tokens,
        }
    )
    md.sort_values("file_name", inplace=True)
    md.reset_index(inplace=True, drop=True)
    return data, md

In [ ]:
# | hide

data_dir = Path(os.getcwd()) / "data" / "dataset_training_sample"

data, md = generate_data(data_dir)

assert sorted(list(data.keys())) == md.file_name.to_list()
assert md.index.to_list() == [i for i in range(md.shape[0])]

2it [00:00, 956.84it/s]


In [ ]:
# | export


def extract_icdar_data(out_dir: TypingText, zip_file: TypingText) -> Tuple[Path, Path]:
    with ZipFile(zip_file, "r") as zip_object:
        zip_object.extractall(path=out_dir)

    # Copy Finnish data
    path = Path(out_dir)
    in_dir = path / "TOOLS_for_Finnish_data"
    inputs = {
        "evaluation": "ICDAR2019_POCR_competition_evaluation_4M_without_Finnish",
        "full": "ICDAR2019_POCR_competition_full_22M_without_Finnish",
        "training": "ICDAR2019_POCR_competition_training_18M_without_Finnish",
    }
    for from_dir, to_dir in inputs.items():
        for in_file in (in_dir / "output" / from_dir).iterdir():
            if in_file.is_file():
                out_file = path / to_dir / "FI" / "FI1" / in_file.name
                shutil.copy2(in_file, out_file)

    # Get paths for train and test data
    train_path = (
        Path(out_dir) / "ICDAR2019_POCR_competition_training_18M_without_Finnish"
    )
    test_path = (
        Path(out_dir) / "ICDAR2019_POCR_competition_evaluation_4M_without_Finnish"
    )

    return train_path, test_path


def get_intermediate_data(
    zip_file: TypingText,
) -> Tuple[Dict[str, Text], pd.DataFrame, Dict[str, Text], pd.DataFrame]:
    """Get the data and metadata for the train and test sets from the zip file."""

    with tempfile.TemporaryDirectory() as tmp_dir:
        train_path, test_path = extract_icdar_data(tmp_dir, zip_file)

        data, md = generate_data(train_path)
        data_test, md_test = generate_data(test_path)

    return (data, md, data_test, md_test)

## Generate input 'sentences'

The following functions can be used to generate sequences of a certain length with possible overlap.


In [ ]:
# | export
def window(iterable, size=2):
    """Given an iterable, return all subsequences of a certain size"""
    i = iter(iterable)
    win = []
    for e in range(0, size):
        try:
            win.append(next(i))
        except StopIteration:
            break
    yield win
    for e in i:
        win = win[1:] + [e]
        yield win

In [ ]:
# | hide

it = [1, 2, 3, 4]
result = [w for w in window(it, 2)]

assert result == [[1, 2], [2, 3], [3, 4]]

In [ ]:
# | export


def _process_sequence(
    key: str,
    i: int,
    res,
    sents: List[List[str]],
    labels: List[List[int]],
    keys: List[str],
    start_tokens: List[int],
    scores: List[float],
    languages: List[str],
) -> Tuple[
    List[List[str]], List[List[int]], List[str], List[int], List[float], List[str]
]:
    ocr = [t.ocr for t in res]
    lbls = [t.label for t in res]
    gs = []
    for t in res:
        if t.gs != "":
            gs.append(t.gs)
    ocr_str = " ".join(ocr)
    gs_str = " ".join(gs)
    ed = edlib.align(ocr_str, gs_str)["editDistance"]
    score = normalized_ed(ed, ocr_str, gs_str)

    if len(ocr_str) > 0:
        sents.append(ocr)
        labels.append(lbls)
        keys.append(key)
        start_tokens.append(i)
        scores.append(score)
        languages.append(key[:2])
    else:
        logger.info(f'Empty sample for text "{key}"')
        logger.info(f"ocr_str: {ocr_str}")
        logger.info(f"start token: {i}")

    return (sents, labels, keys, start_tokens, scores, languages)


def generate_sentences(
    df: pd.DataFrame, data: Dict[str, Text], size: int = 15, step: int = 10
) -> pd.DataFrame:
    """Generate sequences of a certain length and possible overlap"""
    sents: List[List[str]] = []
    labels: List[List[int]] = []
    keys: List[str] = []
    start_tokens: List[int] = []
    scores: List[float] = []
    languages: List[str] = []

    for _, row in tqdm(df.iterrows()):
        key = row.file_name
        tokens = data[key].input_tokens

        # print(len(tokens))
        # print(key)
        for i, res in enumerate(window(tokens, size=size)):
            if i % step == 0:
                (
                    sents,
                    labels,
                    keys,
                    start_tokens,
                    scores,
                    languages,
                ) = _process_sequence(
                    key, i, res, sents, labels, keys, start_tokens, scores, languages
                )
        # Add final sequence
        (sents, labels, keys, start_tokens, scores, languages) = _process_sequence(
            key, i, res, sents, labels, keys, start_tokens, scores, languages
        )

    output = pd.DataFrame(
        {
            "key": keys,
            "start_token_id": start_tokens,
            "score": scores,
            "tokens": sents,
            "tags": labels,
            "language": languages,
        }
    )

    # Adding the final sequence may lead to duplicate rows. Remove those
    output.drop_duplicates(
        subset=["key", "start_token_id"], keep="first", inplace=True, ignore_index=True
    )

    return output

In [ ]:
# | hide

in_file = Path(os.getcwd()) / "data" / "example.txt"
text = process_text(in_file)
data = {"en/en1/1.txt": text}
md = pd.DataFrame(
    {
        "language": ["en"],
        "file_name": ["en/en1/1.txt"],
        "score": [text.score],
        "num_tokens": [len(text.tokens)],
        "num_input_tokens": [len(text.input_tokens)],
    }
)

df = generate_sentences(md, data, size=2, step=2)

assert 2 == df.shape[0]
assert [0, 2] == list(df.start_token_id)

1it [00:00, 2695.57it/s]


In [ ]:
# | export


def process_input_ocr(text: str) -> Text:
    """Generate Text object for OCR input text (without aligned gold standard)"""
    tokens = []
    for match in re.finditer(r"\b\S+(\s|$)", text):
        ocr = match.group().strip()
        gs = ocr
        start = match.start()
        len_ocr = len(ocr)
        label = 0

        tokens.append(
            InputToken(
                ocr=ocr,
                gs=gs,
                start=start,
                len_ocr=len_ocr,
                label=label,
            )
        )
    return Text(text, tokens=[], input_tokens=tokens, score=-1)

In [ ]:
text = process_input_ocr("This is a cxample...")
data = {"ocr_input": text}
md = pd.DataFrame(
    {
        "language": ["?"],
        "file_name": ["ocr_input"],
        "score": [text.score],
        "num_tokens": [len(text.tokens)],
        "num_input_tokens": [len(text.input_tokens)],
    }
)

df = generate_sentences(md, data, size=2, step=2)

assert 2 == df.shape[0]
assert [0, 2] == list(df.start_token_id)
df

1it [00:00, 4275.54it/s]


,key,start_token_id,score,tokens,tags,language
0,ocr_input,0,0.0,"[This, is]","[0, 0]",oc
1,ocr_input,2,0.0,"[a, cxample...]","[0, 0]",oc


In [ ]:
# | hide
import nbdev

nbdev.nbdev_export()